In [ ]:
import openai
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from deep_translator import GoogleTranslator
from langdetect import detect
from langchain_core.documents import Document

# Initialize LLM
llm = ChatOpenAI(
    openai_api_key="",
    temperature=0,
    model_name="gpt-3.5-turbo"
)


In [65]:
translator = GoogleTranslator()

def detect_language(text):
    lang = detect(text)
    print(f"Detected language: {lang}")
    return lang

def translate_text(text, target_lang):
    return GoogleTranslator(source='auto', target=target_lang).translate(text)

In [ ]:
from langchain.document_loaders import TextLoader

# Example document loader (you can customize or load your files)
loader = TextLoader('japanese_text.txt', encoding='utf-8')
documents = loader.load()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

translated_split_docs = []
for doc in split_docs:
    translated_text = GoogleTranslator(source='ja', target='en').translate(doc.page_content)
    translated_split_docs.append(Document(page_content=translated_text, metadata=doc.metadata))

# Generate Embeddings and Create Vector Store
embeddings = OpenAIEmbeddings(openai_api_key="")
vector_store = Chroma.from_documents(translated_split_docs, embeddings)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.document_loaders import UnstructuredPDFLoader


# # Initialize PDF loader
# pdf_loader = UnstructuredPDFLoader('gce-advance-level-exam-2024-sinhala-past-papers-67767260c5c2c.pdf')
# pdf_documents = pdf_loader.load()

# # Split PDF into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# split_pdf_docs = text_splitter.split_documents(pdf_documents)

# # Initialize OpenAI embeddings
# embeddings = OpenAIEmbeddings(openai_api_key="")

# # Create vector store from PDF chunks
# vector_store = Chroma.from_documents(split_pdf_docs, embeddings)


In [68]:
retriever = vector_store.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [69]:

def translate_text(text, target_lang):
    return GoogleTranslator(source='auto', target=target_lang).translate(text)

def detect_language(text):
    try:
        return detect(text)
    except Exception:
        return "en"  # fallback if detection fails

def multi_language_qa(user_question):
    try:
        # Detect user's language
        input_lang = detect_language(user_question)

        # Translate question to English for RAG
        question_in_english = translate_text(user_question, "en")
        print(f"Translated Question to English: {question_in_english}")

        # Get answer from QA system
        answer_in_english = qa_chain.run(question_in_english)
        print(f"Answer in English: {answer_in_english}")

        # Always translate answer back to user's language
        translated_answer = translate_text(answer_in_english, input_lang)
        return translated_answer

    except Exception as e:
        return f"Error during QA: {str(e)}"


In [71]:
question = "¿Cuál es el significado de este texto?"
result = multi_language_qa(question) 
print("Answer:", result)


Translated Question to English: What is the meaning of this text?
Answer in English: The text is a repeated message in both English and Japanese, inviting the reader to learn about artificial intelligence and machine learning.
Answer: El texto es un mensaje repetido tanto en inglés como en japonés, invitando al lector a aprender sobre inteligencia artificial y aprendizaje automático.
